## Video Augmentation for final project

In [16]:
from PIL import Image, ImageSequence

from pathlib import Path
import sys
sys.path.append(str(Path('/home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/vidaug/augmentors').parent.parent))

import vidaug.augmentors as va
import imageio
import cv2
import numpy as np
import pandas as pd

import moviepy.editor as mp



In [17]:
# Function that loads .mp4 video

def load_video(path, max_frames=0):
    cap = cv2.VideoCapture(path)
    # getting total frames and fps 
    #total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = round (cap.get(cv2.CAP_PROP_FPS),0)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames), fps

In [18]:
# Function that converts .mp4 to gif
def to_gif(video_path_mp4, video_path_save_gif, max_frames):
    images, fps = load_video(video_path_mp4, max_frames)
    converted_images = images.astype(np.uint8)
    imageio.mimsave(video_path_save_gif, converted_images, fps=fps)
    return 

In [19]:
# Function that loads .gif
def gif_loader(path, modality="RGB"):
    frames = []
    with open(path, 'rb') as f:
        with Image.open(f) as video:
            index = 1
            for frame in ImageSequence.Iterator(video):
                frames.append(frame.convert(modality))
                index += 1
            video.close()
        f.close()
    return frames


Defining the types of data augmentation that have been applied

In [20]:
# Initializing list of sequences, wich one is a type of data aug
list_seq = []

sometimes = lambda aug: va.Sometimes(1, aug) # Used to apply augmentor with 100% probability

# FLIP
seq = va.Sequential([ # randomly rotates the video with a degree randomly choosen from [-10, 10]  
    sometimes(va.HorizontalFlip()), # horizontally flip the video with 100% probability
    ])
seq1 = va.Sequential([   
    sometimes(va.VerticalFlip()), # vertically flip the video with 100% probability
    ])

# AFFINE
seq2 = va.Sequential([  
sometimes(va.RandomRotate(degrees=45)), # Rotate the video 45 degrees  with 100% probability
])
seq3 = va.Sequential([   
sometimes(va.RandomTranslate(x=500, y=500)), # Moves the screen
])

# GEOMETRIC
seq4 = va.Sequential([  
sometimes(va.PiecewiseAffineTransform(displacement=10, displacement_kernel=0.5, displacement_magnification=5)),
])

# INTENSITY
seq5 = va.Sequential([  
sometimes(va.InvertColor()), 
])
seq6= va.Sequential([  
sometimes(va.Add(value=155)), # value > 255 or value < -255 
])
seq7= va.Sequential([  
sometimes(va.Multiply(value=7)), # value > 0
])
seq8= va.Sequential([  
sometimes(va.Pepper(ratio=3)), # Smaller the ratio, higher the number of black pixels.
])
seq9= va.Sequential([  
sometimes(va.Salt(ratio=4)), # Smaller the ratio, higher the number of white pixels.
])
#TEMPORAL
seq10= va.Sequential([  
sometimes(va.Downsample(ratio=0.5)), # Downsampling ratio in [0.0 <= ratio <= 1.0]
])

# Appending every data aug to the list
list_seq.append(seq)
list_seq.append(seq1)
list_seq.append(seq2)
list_seq.append(seq3)
list_seq.append(seq4)
list_seq.append(seq5)
list_seq.append(seq6)
list_seq.append(seq7)
list_seq.append(seq8)
list_seq.append(seq9)
list_seq.append(seq10)

Loading dataframe for raw videos

In [22]:
df_videos_to_aug = pd.read_csv('/home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/CSV/dataset_to_aug_LOCAL.csv',dtype={"shot": str})
df_videos_to_aug

,video_name,tag,movie,shot
0,/home/danield/AnyoneAI/Curso/final_project/vid...,Pull,tt1376168,0034
1,/home/danield/AnyoneAI/Curso/final_project/vid...,Push,tt1828968,0050
2,/home/danield/AnyoneAI/Curso/final_project/vid...,Push,tt0108289,0022
3,/home/danield/AnyoneAI/Curso/final_project/vid...,Push,tt0108289,0042
4,/home/danield/AnyoneAI/Curso/final_project/vid...,Pull,tt2349952,0053
...,...,...,...,...
848,/home/danield/AnyoneAI/Curso/final_project/vid...,Pull,tt5082670,0005
849,/home/danield/AnyoneAI/Curso/final_project/vid...,Pull,tt5085688,0007
850,/home/danield/AnyoneAI/Curso/final_project/vid...,Pull,tt5967998,0022
851,/home/danield/AnyoneAI/Curso/final_project/vid...,Pull,tt5967998,0063


In [23]:
# Script for pass server video address to local address

new_addr_list=[]

for idx_videos, video_to_aug in df_videos_to_aug.iterrows():
    new_addr = str('/home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/to_data_aug/'+ video_to_aug.movie + '_' + video_to_aug.shot + '.mp4')
    new_addr_list.append(new_addr)

df_videos_to_aug['video_name'] = new_addr_list

#df_videos_to_aug.to_csv('/home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/CSV/dataset_to_aug_LOCAL.csv', index = False)


VIDEO AUGMENTATION

In [24]:
# VIDEO AUGMENTATION

# max lenght of each video
max_frames = 20

path_temp_folder = '/home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/shot-type-dataset/temp/'
# path of .mp4 to augment
#video_path_mp4 = '/home/app/src/vidaug/videos/shot_0018.mp4'

#path to intermediate .gif
video_path_save_gif = '/home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/shot-type-dataset/temp/transitional.gif'
#path to final .mp4 videos
path_to_save_aug_videos = '/home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/'

video_aug_name = []
tag = []
# Getting each video of theirs df video_name and processing
for idx_videos, video_to_aug in df_videos_to_aug.iterrows():
    
    # Converting .mp4 to gif
    to_gif(video_to_aug['video_name'], video_path_save_gif, max_frames = max_frames)
    # loading .gif
    frames = gif_loader(video_path_save_gif)
    print(f'processing video {idx_videos}/{len(df_videos_to_aug.video_name)}')
 
    # applying every transformation over the videos
    for idx_seq, iter_seq in enumerate(list_seq): 
        video_aug = iter_seq(frames)
        # save augmentad frames as gif 
        video_aug[0].save(path_temp_folder + f'aug_{idx_seq}' + ".gif", save_all=True, append_images=video_aug[1:int(max_frames)], loop=0)
        # Converting .gif augmented videos as final .mp4
        clip = mp.VideoFileClip(path_temp_folder + f'aug_{idx_seq}' + ".gif")
        clip.write_videofile(path_to_save_aug_videos + df_videos_to_aug.movie[idx_videos]+ '_' + \
                             df_videos_to_aug.shot[idx_videos]+ f'_aug_{idx_seq}' + ".mp4")

        # Building the list to the new movies df
        video_aug_name.append(df_videos_to_aug.movie[idx_videos]+ '_' +df_videos_to_aug.shot[idx_videos]+ f'_aug_{idx_seq}' + ".mp4")
        tag.append(df_videos_to_aug.tag[idx_videos])
        
# Building new movies df
data = list(zip(video_aug_name, tag))  
columns = ['video_aug_name', 'tag']  
        
df_trailer_aug = pd.DataFrame(data, columns=columns)      

    

processing video 0/853
Moviepy - Building video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt1376168_0034_aug_0.mp4.
Moviepy - Writing video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt1376168_0034_aug_0.mp4



Moviepy - Done !
Moviepy - video ready /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt1376168_0034_aug_0.mp4


Moviepy - Building video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt1376168_0034_aug_1.mp4.
Moviepy - Writing video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt1376168_0034_aug_1.mp4



Moviepy - Done !
Moviepy - video ready /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt1376168_0034_aug_1.mp4


Moviepy - Building video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt1376168_0034_aug_2.mp4.
Moviepy - Writing video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt1376168_0034_aug_2.mp4



...

Moviepy - Done !
Moviepy - video ready /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_2.mp4
Moviepy - Building video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_3.mp4.
Moviepy - Writing video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_3.mp4



Moviepy - Done !
Moviepy - video ready /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_3.mp4
Moviepy - Building video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_4.mp4.
Moviepy - Writing video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_4.mp4



Moviepy - Done !
Moviepy - video ready /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_4.mp4
Moviepy - Building video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_5.mp4.
Moviepy - Writing video /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_5.mp4



Moviepy - Done !
Moviepy - video ready /home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/data/trailer_aug/tt0071615_0028_aug_5.mp4


KeyboardInterrupt: 

In [ ]:

df_trailer_aug.to_csv('/home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/CSV/df_trailer_aug.csv', index = False)


In [ ]:
df_trailer_aug

,video_aug_name,tag
0,tt1376168_0034_aug_0.mp4,Pull
1,tt1376168_0034_aug_1.mp4,Pull
2,tt1376168_0034_aug_2.mp4,Pull
3,tt1376168_0034_aug_3.mp4,Pull
4,tt1376168_0034_aug_4.mp4,Pull
...,...,...
8525,tt6210456_0021_aug_5.mp4,Pull
8526,tt6210456_0021_aug_6.mp4,Pull
8527,tt6210456_0021_aug_7.mp4,Pull
8528,tt6210456_0021_aug_8.mp4,Pull
